# Time-optimal path planning
This notebook handles standard code for the computation of time-optimal paths in windfields

In [1]:
import os
import sys
import time
import numpy as np
from math import atan, cos, sin
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])

from mermoz.feedback import TargetFB
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.problem import MermozProblem
from mermoz.model import ZermeloGeneralModel
from mermoz.solver_rp import SolverRP
from mermoz.stoppingcond import TimedSC, DistanceSC
from mermoz.wind import DiscreteWind
from mermoz.misc import *
from mdisplay.geodata import GeoData

from solver_utils import select_wind

%load_ext autoreload
%autoreload

### Problem definition

Initial point, target, UAV airspeed

In [2]:
# Scale factor
sf = 1.

# Initial point
x_init = sf * np.array((0.6, 0.6))

# Target
x_target = sf * np.array((2.4, 2.4))

# Windfield boundaries
bl = sf * np.array((0.5, 0.5))
tr = sf * np.array((2.5, 2.5))

# UAV airspeed in m/s
v_a = 0.05

case_name = 'double-gyre-kularatne2016_rp'

Set up the problem

In [3]:
# Prepare output directory
output_dir = f'/home/bastien/Documents/work/mermoz/output/example_solver_{case_name}'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create a file manager to dump problem data
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)
mdfm.clean_output_dir()

# Problem coordinates
# coords = COORD_GCS
coords = COORD_CARTESIAN

# Problem wind data
from mermoz.wind import DoubleGyreWind
total_wind = DoubleGyreWind(0.5, 0.5, 2., 2., pi * 0.02)
def domain(x):
    return bl[0] < x[0] < tr[0] and bl[1] < x[1] < tr[1]

# Creates the cinematic model
zermelo_model = ZermeloGeneralModel(v_a, coords=coords)
zermelo_model.update_wind(total_wind)

# Creates the navigation problem on top of the previous model
mp = MermozProblem(zermelo_model, coords=coords, autodomain=False, domain=domain)

### Solver parameters

In [4]:
nx_rft = 51
ny_rft = 51
nt_rft = 10
solver_rp = SolverRP(mp, x_init, x_target, nx_rft, ny_rft, nt_rft)

Solve problem

In [5]:
t_start = time.time()
solver_rp.solve()
t_end = time.time()
time_pmp = t_end - t_start

Tracking reachability front (51x51x10)... 
tol =

    0.0293

	76 steps in 0.39 seconds from 0 to 11.3137
	76 steps in 0.38 seconds from 11.3137 to 22.6274
	76 steps in 0.55 seconds from 22.6274 to 33.9411
	76 steps in 0.53 seconds from 33.9411 to 45.2548
	76 steps in 0.36 seconds from 45.2548 to 56.5685
	76 steps in 0.16 seconds from 56.5685 to 67.8823
	76 steps in 0.36 seconds from 67.8823 to 79.196
	76 steps in 0.55 seconds from 79.196 to 90.5097
	76 steps in 0.53 seconds from 90.5097 to 101.823

Total execution time 4.43 seconds
Done (9.572 s)
Shooting -4.2
Shooting -2.0
Shooting 0.2
Shooting 2.5
Shooting 4.7
Shooting 6.9
Shooting 9.1
Shooting 11.4
Shooting 13.6
Shooting 15.8
0.77117, 5.80
1.05993, 3.58
1.01442, 8.03
1.47573, 12.47
1.32445, 1.36
1.43975, -0.86
1.33401, 10.25
1.53951, -3.09
1.67189, 14.69
It   1 - Shoot  5.8 - Depth   1 - Crit 0.5, 20% - Branching LU
It   2 - Shoot  6.4 - Depth   2 - Crit 0.7, 26% - Branching  L
It   3 - Shoot  6.1 - Depth   3 - Crit 0.6, 24% - Bran

In [6]:
sc = DistanceSC(lambda x: np.linalg.norm(x - x_target), np.linalg.norm(x_init - x_target)/100.)
T = distance(x_init, x_target, coords='cartesian') / v_a
nt_pmp = 1000
sc = TimedSC(T)
from mermoz.feedback import FixedHeadingFB, ConstantFB

mp.load_feedback(FixedHeadingFB(mp.model.wind, v_a, 0., mp.coords))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(ConstantFB(0.))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

Dump everything to output directory

In [7]:
nx = 51
ny = 51
mdfm.dump_wind(total_wind, nx=nx, ny=ny, bl=bl, tr=tr)
mdfm.dump_trajs(mp.trajs)
mdfm.dump_trajs(solver_rp.mp_pmp.trajs)
solver_rp.rft.dump_rff(output_dir)
ps = ParamsSummary({}, output_dir)
ps.load_from_solver(solver_rp)
ps.add_param('bl_wind', tuple(bl))
ps.add_param('tr_wind', tuple(tr))
#ps.add_param('nx_wind', nx)
#ps.add_param('ny_wind', ny)
#ps.add_param('nt_pmp', nt_pmp)
#ps.add_param('pmp_time', time_pmp)
ps.dump()

<KeysViewHDF5 []>
0
<KeysViewHDF5 ['0', '1']>
2
